{/* cspell:ignore IIZII, XIZZZ, accum */}

# Primitives with REST API

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```

```
</details>

<LegacyContent>
<Admonition type="note">
This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/primitives-rest-api)
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note">
This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/primitives-rest-api)
</Admonition>
</CloudContent>

The steps in this topic describe how to run and configure primitive workloads with REST API, and demonstrate how to invoke them in any program of your choice.

<Admonition type="note">
    This documentation utilizes the Python `requests` module to demonstrate the Qiskit Runtime REST API. However, this workflow can be executed using any language or framework that supports working with REST APIs. Refer to the [API reference documentation](/docs/api/runtime/tags/jobs) for details.
</Admonition>

<span id="start-estimator"></span>
## Estimator primitive with REST API

### 1. Initialize the account

Because Qiskit Runtime Estimator is a managed service, you first need to initialize your account. You can then select the device you want to use to calculate the expectation value.
<LegacyContent>
Find details on how to initialize your account, view available backends, and invalidate tokens in this [topic](./setup-channel#set-up-to-use-ibm-quantum-platform-with-rest-api).
</LegacyContent>
<CloudContent>
Find details on how to initialize your account, view available backends, and invalidate tokens in this [topic](./cloud-setup#set-up-to-use-ibm-quantum-platform-with-rest-api).
</CloudContent>
### 2. Create a QASM circuit

You need at least one circuit as the input to the Estimator primitive.

Define a QASM quantum circuit. For example:

```python
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[2];
creg c[2];
x q[0];
cx q[0], q[1];
c[0] = measure q[0];
c[1] = measure q[1];
'''
```

The following code snippets assume that the `qasm_string` has been transpiled to a new string `resulting_qasm`. Find detailed instructions on how to use the Qiskit Transpiler Service API in this [topic](./transpile-rest-api).

### 3. Run the quantum circuit using the Estimator V2 API


<Admonition type="note">
    The following jobs use [Qiskit Runtime V2 primitives](/docs/migration-guides/v2-primitives). Both `SamplerV2` and `EstimatorV2` take one or more primitive unified blocs (PUBs) as the input. Each PUB is a tuple that contains one circuit and the data broadcasted to that circuit, which can be multiple observables and parameters. Each PUB returns a result.
</Admonition>

<LegacyContent>
```python
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
auth_id = "Bearer <YOUR_API_TOKEN>"
backend = "<BACKEND_NAME>"
headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id,
    }
job_input = {
    'program_id': 'estimator',
    "backend": backend,
    "hub": "hub1",
    "group": "group1",
    "project": "project1",
    "params": {
        "pubs": [ #primitive unified blocs (PUBs) containing one circuit each. c.f. https://docs.quantum.ibm.com/migration-guides/v2-primitives
            [resulting_qasm, # QASM circuit
             {"IIZII": 1, "XIZZZ": 2.3}, # Observable
             None # parameter values
             ]]
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```
</LegacyContent>


<CloudContent>
```python
import requests

url = 'https://quantum.cloud.ibm.com/v1/jobs'
auth_id = "Bearer <YOUR_BEARER_TOKEN>"
crn = "<SERVICE-CRN>"
backend = "<BACKEND_NAME>"

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id,
    'Service-CRN': crn
    }

job_input = {
    'program_id': 'estimator',
    "backend": backend,
    "params": {
        "pubs": [ #primitive unified blocs (PUBs) containing one circuit each.
            [resulting_qasm, # QASM circuit
             {"IIZII": 1, "XIZZZ": 2.3}, # Observable
             None # parameter values
             ]]
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```
</CloudContent>


### 4. Check job status and get results

Next, pass the `job_id` to the API:

```python
response_status_singlejob= requests.get(url+'/'+job_id, headers=headers)
response_status_singlejob.json().get('state')
```
Output

```text
>>> Job ID: 58223448-5100-4dec-a47a-942fb30edcad
>>> Job Status: JobStatus.RUNNING
```

Get job results:
```python
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

res_dict=response_result.json()

estimator_result=res_dict['results']
print(estimator_result)
```
Output
```text
[{'data': {'evs': 0.7428980350102542, 'stds': 0.029884014518789213, 'ensemble_standard_error': 0.03261147170624149}, 'metadata': {'shots': 10016, 'target_precision': 0.01, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}}]
```

### 5. Work with Runtime options

Error mitigation techniques allow users to mitigate circuit errors by modeling the device noise at the time of execution. This typically results in quantum pre-processing overhead related to model training, and classical post-processing overhead to mitigate errors in the raw results by using the generated model.

The error mitigation techniques built in to primitives are advanced resilience options. To specify these options, use the `resilience_level` option when submitting your job.

The following examples demonstrate the default options for dynamical decoupling, twirling, and TREX + ZNE. Find more options and further details in the [Error mitigation and suppression techniques](./error-mitigation-and-suppression-techniques) topic.

<LegacyContent>
<Tabs>

   <TabItem value="TREX + ZNE" label="TREX + ZNE">

  ```python
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
auth_id = "Bearer <YOUR_API_TOKEN>"
backend = "<BACKEND_NAME>"

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id
    }
job_input = {
    'program_id': 'estimator',
    "backend": backend,
    "hub": "hub1",
    "group": "group1",
    "project": "project1",
    "params": {
        "pubs": [ #primitive unified blocs (PUBs) containing one circuit each.
            [resulting_qasm, # QASM circuit
             {"IIZII": 1, "XIZZZ": 2.3}, # Observable
             None # parameter values
             ]],
        "options": {
            "resilience": {
              "measure_mitigation": True,
              "zne_mitigation": True,
              "zne": {
                  "extrapolator":["exponential", "linear"],
                  "noise_factors":[1, 3, 5],
              },
          }
        }
    }
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```

  </TabItem>

  <TabItem value="Dynamical Decoupling" label="Dynamical Decoupling">

  ```python
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
auth_id = "Bearer <YOUR_API_TOKEN>"
backend = '<BACKEND_NAME>'

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id
    }
job_input = {
    'program_id': 'estimator',
    "backend": backend,
    "hub": "hub1",
    "group": "group1",
    "project": "project1",
    "params": {
        "pubs": [ #primitive unified blocs (PUBs) containing one circuit each.
            [resulting_qasm, # QASM circuit
             {"IIZII": 1, "XIZZZ": 2.3}, # Observable
             None # parameter values
             ]]
        "options": {
            "dynamical_decoupling": {
                "enable": True,
                "sequence_type": 'XpXm',
                "extra_slack_distribution": 'middle',
                "scheduling_method": 'alap',
            },
        },
    }
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```

  </TabItem>

  <TabItem value="Twirling" label="Twirling">

  ```python
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
auth_id = "Bearer <YOUR_API_TOKEN>"
backend = '<BACKEND_NAME>'

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id
    }
job_input = {
    'program_id': 'estimator',
    "backend": backend,
    "hub": "hub1",
    "group": "group1",
    "project": "project1",
    "params": {
        "pubs": [ #primitive unified blocs (PUBs) containing one circuit each.
            [resulting_qasm, # QASM circuit
             {"IIZII": 1, "XIZZZ": 2.3}, # Observable
             None # parameter values
             ]]
        "options": {
            "twirling": {
                "enable_gates": True,
                "enable_measure": True,
                "num_randomizations": "auto",
                "shots_per_randomization": "auto",
                "strategy": "active-accum",
                },
        },
    }
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```
  </TabItem>
</Tabs>
</LegacyContent>

<CloudContent>
<Tabs>

   <TabItem value="TREX + ZNE" label="TREX + ZNE">

  ```python
import requests

url = 'https://quantum.cloud.ibm.com/v1/jobs'
auth_id = "Bearer <YOUR_BEARER_TOKEN>"
crn = "<SERVICE-CRN>"
backend = "BACKEND_NAME"

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id,
    'Service-CRN': crn
    }
job_input = {
    'program_id': 'estimator',
    "backend": backend,
    "params": {
        "pubs": [ #primitive unified blocs (PUBs) containing one circuit each
            [resulting_qasm, # QASM circuit
             {"IIZII": 1, "XIZZZ": 2.3}, # Observable
             None # parameter values
             ]]
        "options": {
            "resilience": {
              "measure_mitigation": True,
              "zne_mitigation": True,
              "zne": {
                  "extrapolator":["exponential", "linear"],
                  "noise_factors":[1, 3, 5],
              },
          },
        },
    }
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```

  </TabItem>

  <TabItem value="Dynamical Decoupling" label="Dynamical Decoupling">

  ```python
import requests

url = 'https://quantum.cloud.ibm.com/v1/jobs'
auth_id = "Bearer <YOUR_BEARER_TOKEN>"
crn = "<SERVICE-CRN>"
backend = "BACKEND_NAME"

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id,
    'Service-CRN': crn
    }
job_input = {
    'program_id': 'estimator',
    "backend": backend,
    "params": {
        "pubs": [ #primitive unified blocs (PUBs) containing one circuit each
            [resulting_qasm, # QASM circuit
             {"IIZII": 1, "XIZZZ": 2.3}, # Observable
             None # parameter values
             ]]
        "options": {
            "dynamical_decoupling": {
                "enable": True,
                "sequence_type": 'XpXm',
                "extra_slack_distribution": 'middle',
                "scheduling_method": 'alap',
            },
        },
    }
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```

  </TabItem>

  <TabItem value="Twirling" label="Twirling">

  ```python
import requests

url = 'https://quantum.cloud.ibm.com/v1/jobs'
auth_id = "Bearer <YOUR_BEARER_TOKEN>"
crn = "<SERVICE-CRN>"
backend = "BACKEND_NAME"

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id,
    'Service-CRN': crn
    }
job_input = {
    'program_id': 'estimator',
    "backend": backend,
    "params": {
        "pubs": [ #primitive unified blocs (PUBs) containing one circuit each
            [resulting_qasm, # QASM circuit
             {"IIZII": 1, "XIZZZ": 2.3}, # Observable
             None # parameter values
             ]]
        "options": {
            "twirling": {
                "enable_gates": True,
                "enable_measure": True,
                "num_randomizations": "auto",
                "shots_per_randomization": "auto",
                "strategy": "active-accum",
                },
        },
    }
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```


  </TabItem>
</Tabs>
</CloudContent>


<span id="start-sampler"></span>
## Sampler primitive with REST API

### 1. Initialize the account

Because Qiskit Runtime Sampler is a managed service, you first need to initialize your account. You can then select the device you want to use to run your calculations on.

<LegacyContent>
Find details on how to initialize your account, view available backends, and invalidate tokens in this [topic](./setup-channel#set-up-to-use-ibm-quantum-platform-with-rest-api).
</LegacyContent>
<CloudContent>
Find details on how to initialize your account, view available backends, and invalidate tokens in this [topic](./cloud-setup#set-up-to-use-ibm-quantum-platform-with-rest-api).
</CloudContent>

### 2. Create a QASM circuit

You need at least one circuit as the input to the Sampler primitive.


Define a QASM quantum circuit:

```python
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[2];
creg c[2];
x q[0];
cx q[0], q[1];
c[0] = measure q[0];
c[1] = measure q[1];
'''
```


The code snippets given below assume that the `qasm_string` has been transpiled to a new string `resulting_qasm`. Find detailed instructions on how to use the Qiskit Transpiler Service API in this [topic](./transpile-rest-api).


### 3. Run the quantum circuit using Sampler V2 API


<Admonition type="note">
    The jobs below use [Qiskit Runtime V2 primitives](/docs/migration-guides/v2-primitives). Both `SamplerV2` and `EstimatorV2` take one or more primitive unified blocs (PUBs) as the input. Each PUB is a tuple that contains one circuit and the data broadcasted to that circuit, which can be multiple observables and parameters. Each PUB returns a result.
</Admonition>

<LegacyContent>
```python
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
auth_id = "Bearer <YOUR_API_TOKEN>"
backend = '<BACKEND_NAME>'

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id
    }
job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "hub": "hub1",
    "group": "group1",
    "project": "project1",
    "params": {
        "pubs": [[resulting_qasm],[resulting_qasm,None,500]] # primitive unified blocs (PUBs) containing one circuit each.
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```
</LegacyContent>

<CloudContent>
```python
import requests

url = 'https://quantum.cloud.ibm.com/v1/jobs'
auth_id = "Bearer <YOUR_BEARER_TOKEN>"
crn = "<SERVICE-CRN>"
backend = "<BACKEND_NAME>"

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id,
    'Service-CRN': crn
    }
job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "params": {
        "pubs": [[resulting_qasm],[resulting_qasm,None,500]] # primitive unified blocs (PUBs) containing one circuit each.
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```
</CloudContent>


### 4. Check job status and get results

Next, pass the `job_id` to the API:

```python
response_status_singlejob= requests.get(url+'/'+job_id, headers=headers)
response_status_singlejob.json().get('state')
```
Output

```text
>>> Job ID: 58223448-5100-4dec-a47a-942fb30edced
>>> Job Status: JobStatus.RUNNING
```

Get job results:
```python
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

res_dict=response_result.json()

# Get results for the first PUB
counts=res_dict['results'][0]['data']['c']['samples']

print(counts[:20])
```
Output
```text
['0x3', '0x0', '0x2', '0x1', '0x0', '0x3', '0x0', '0x3', '0x1', '0x2', '0x2', '0x0', '0x2', '0x0', '0x3', '0x3', '0x2', '0x0', '0x1', '0x0']
```

### 5. Work with Runtime options

Error mitigation techniques allow users to mitigate circuit errors by modeling the device noise at the time of execution. This typically results in quantum pre-processing overhead related to model training, and classical post-processing overhead to mitigate errors in the raw results by using the generated model.

The error mitigation techniques built in to primitives are advanced resilience options. To specify these options, use the `resilience_level` option when submitting your job.
Sampler V2 does not support specifying resilience levels. However, you can turn on or off individual error mitigation / suppression methods.

The following examples demonstrate the default options for dynamical decoupling and twirling. Find more options and further details in the [Error mitigation and suppression techniques](./error-mitigation-and-suppression-techniques) topic.

<LegacyContent>
<Tabs>

  <TabItem value="Dynamical Decoupling" label="Dynamical Decoupling">

  ```python
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
auth_id = "Bearer <YOUR_API_TOKEN>"
backend = '<BACKEND_NAME>'

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id
    }
job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "hub": "hub1",
    "group": "group1",
    "project": "project1",
    "params": {
        "pubs": [[resulting_qasm]], # primitive unified blocs (PUBs) containing one circuit each.
        "options": {
            "dynamical_decoupling": {
                "enable": True,
                "sequence_type": 'XpXm',
                "extra_slack_distribution": 'middle',
                "scheduling_method": 'alap',
            },
        },
    }
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```

  </TabItem>

  <TabItem value="Twirling" label="Twirling">

  ```python
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
auth_id = "Bearer <YOUR_API_TOKEN>"
backend = '<BACKEND_NAME>'

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id
    }
job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "hub": "hub1",
    "group": "group1",
    "project": "project1",
    "params": {
        "pubs": [[resulting_qasm]] # primitive unified blocs (PUBs) containing one circuit each.
        "options": {
            "twirling": {
                "enable_gates": True,
                "enable_measure": True,
                "num_randomizations": "auto",
                "shots_per_randomization": "auto",
                "strategy": "active-accum",
                },
        },
    }
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```


  </TabItem>
</Tabs>
</LegacyContent>

<CloudContent>
<Tabs>

  <TabItem value="Dynamical Decoupling" label="Dynamical Decoupling">

  ```python
import requests

url = 'https://quantum.cloud.ibm.com/v1/jobs'
auth_id = "Bearer <YOUR_BEARER_TOKEN>"
crn = "<SERVICE-CRN>"
backend = "<BACKEND_NAME>"

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id,
    'Service-CRN': crn
    }
job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "params": {
        "pubs": [[resulting_qasm]], # primitive unified blocs (PUBs) containing one circuit each.
        "options": {
            "dynamical_decoupling": {
                "enable": True,
                "sequence_type": 'XpXm',
                "extra_slack_distribution": 'middle',
                "scheduling_method": 'alap',
            },
        },
    }
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```

  </TabItem>

  <TabItem value="Twirling" label="Twirling">

  ```python
import requests

url = 'https://quantum.cloud.ibm.com/v1/jobs'
auth_id = "Bearer <YOUR_BEARER_TOKEN>"
crn = "<SERVICE-CRN>"
backend = "<BACKEND_NAME>"

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id,
    'Service-CRN': crn
    }
job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "params": {
        "pubs": [[resulting_qasm]], # primitive unified blocs (PUBs) containing one circuit each.
        "options": {
            "twirling": {
                "enable_gates": True,
                "enable_measure": True,
                "num_randomizations": "auto",
                "shots_per_randomization": "auto",
                "strategy": "active-accum",
                },
        },
    }
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")
```


  </TabItem>
</Tabs>
</CloudContent>

<span id="start-sampler-parms"></span>
## Sampler primitive with REST API and parameterized circuits

### 1. Initialize the account

Because Qiskit Runtime is a managed service, you first need to initialize your account. You can then select the device you want to use to run your calculations on.

<LegacyContent>
Find details on how to initialize your account, view available backends, and invalidate tokens in this [topic](./setup-channel#set-up-to-use-ibm-quantum-platform-with-rest-api).
</LegacyContent>
<CloudContent>
Find details on how to initialize your account, view available backends, and invalidate tokens in this [topic](./cloud-setup#set-up-to-use-ibm-quantum-platform-with-rest-api).
</CloudContent>

### 2. Define parameters

```python
import requests
import qiskit_ibm_runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.qasm3 import dumps
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit import transpile

service = QiskitRuntimeService(channel='ibm_quantum')
backend = service.backend("<your selected backend>")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

theta = Parameter('theta')
phi = Parameter('phi')
parameter_values = {'theta': 1.57, 'phi': 3.14}   # In case we want to pass a dictionary
```


### 3. Create a quantum circuit and add parameterized gates

```python
qc = QuantumCircuit(2)

# Add parameterized gates
qc.rx(theta, 0)
qc.ry(phi, 1)
qc.cx(0, 1)
qc.measure_all()

# Draw the original circuit
qc.draw('mpl')

# Get an ISA circuit
isa_circuit = pm.run(qc)
```

### 4. Generate QASM 3 code

```python
qasm_str = dumps(isa_circuit)
print("Generated QASM 3 code:")
print(qasm_str)
```

### 5. Run the quantum circuit using Sampler V2 API


<Admonition type="note">
    The following jobs use Qiskit Runtime V2 [primitives](/docs/guides/primitives). Both [`SamplerV2`](/docs/api/qiskit-ibm-runtime/sampler-v2) and [`EstimatorV2`](/docs/api/qiskit-ibm-runtime/estimator-v2) take one or more [primitive unified blocs (PUBs)](/docs/guides/primitive-input-output#pubs) as the input. Each PUB is a tuple that contains one circuit and the data broadcasted to that circuit, which can be multiple observables and parameters. Each PUB returns a result.
</Admonition>

<LegacyContent>
```python
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
auth_id = "Bearer <YOUR_API_TOKEN>"
backend = '<BACKEND_NAME>'

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id
    }

job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "hub": "hub1",
    "group": "group1",
    "project": "project1",
    "params": {
        # Choose one option: direct parameter transfer or through a dictionary
        #"pubs": [[qasm_str,[1,2],500]], # primitive unified blocs (PUBs) containing one circuit each.
        "pubs": [[qasm_str,parameter_values,500]], # primitive unified blocs (PUBs) containing one circuit each.
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print(f"Job created: {response.text}")
else:
    print(f"Error: {response.status_code}")
```

```python
print(response.text)
```
</LegacyContent>

<CloudContent>
```python
import requests

url = 'https://quantum.cloud.ibm.com/v1/jobs'
auth_id = "Bearer <YOUR_BEARER_TOKEN>"
crn = "<SERVICE-CRN>"
backend = "<BACKEND_NAME>"

headers = {
    'Content-Type': 'application/json',
    'Authorization':auth_id,
    'Service-CRN': crn
    }

job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "params": {
        # Choose one option: direct parameter transfer or through a dictionary
        #"pubs": [[qasm_str,[1,2],500]], # primitive unified blocs (PUBs) containing one circuit each.
        "pubs": [[qasm_str,parameter_values,500]], # primitive unified blocs (PUBs) containing one circuit each.
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print(f"Job created: {response.text}")
else:
    print(f"Error: {response.status_code}")
```

```python
print(response.text)
```
</CloudContent>

### 6. Check job status and get results

Next, pass the `job_id` to the API:

```python
response_status_singlejob = requests.get(f"{url}/{job_id}", headers=headers)
response_status_singlejob.json().get('state')
```

Output

```text
{'status': 'Completed'}
```

Get job results:

```python
response_result = requests.get(f"{url}/{job_id}/results", headers=headers)

res_dict=response_result.json()

# Get results for the first PUB
counts=res_dict['results'][0]['data']['c']['samples']

print(counts[:20])
```

Output

```text
['0x1', '0x2', '0x1', '0x2', '0x1', '0x2', '0x0', '0x2', '0x1', '0x1', '0x2', '0x2', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1']
```

## Next steps
<LegacyContent>
<Admonition type="tip" title="Recommendations">
    - There are several ways to run workloads, depending on your needs: job mode, session mode and batch mode. Learn how to work with session mode and batch mode in the [execution modes topic](./execution-modes-rest-api).
    - Learn how to [initialize your account](./setup-channel#set-up-to-use-ibm-quantum-platform-with-rest-api) with REST API.
    - Learn how to [transpile circuits using REST API](./transpile-rest-api).
    - Read [Migrate to V2 primitives](/docs/migration-guides/v2-primitives).
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile) section.
    - [Migrate to the Qiskit Runtime V2 primitives.](/docs/migration-guides/v2-primitives)
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="tip" title="Recommendations">
    - There are several ways to run workloads, depending on your needs: job mode, session mode, and batch mode. Learn how to work with session mode and batch mode in the [execution modes topic](/docs/guides/execution-modes-rest-api). Note that Open Plan users cannot submit session jobs.
    - Learn how to [initialize your account](./cloud-setup#set-up-to-use-ibm-quantum-platform-with-rest-api) with REST API.
    - Learn how to [transpile circuits using REST API](./transpile-rest-api).
    - Read [Migrate to V2 primitives](/docs/migration-guides/v2-primitives).
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile) section.
    - [Migrate to the Qiskit Runtime V2 primitives.](/docs/migration-guides/v2-primitives)
</Admonition>
</CloudContent>